In [1]:
from games_setup import *
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
import os
import networkx as nx

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
# following results from run_statistics
multimulti_errors = ['BIOMD0000000471_url.xml', 'BIOMD0000000468_url.xml',
'BIOMD0000000190_url.xml', 'BIOMD0000000572_url.xml',
'BIOMD0000000051_url.xml', 'BIOMD0000000479_url.xml',
'BIOMD0000000469_url.xml', 'BIOMD0000000175_url.xml',
'BIOMD0000000247_url.xml', 'BIOMD0000000049_url.xml',
'BIOMD0000000143_url.xml', 'BIOMD0000000470_url.xml',
'BIOMD0000000235_url.xml', 'BIOMD0000000052_url.xml',
'BIOMD0000000503_url.xml', 'BIOMD0000000426_url.xml',
'BIOMD0000000105_url.xml', 'BIOMD0000000167_url.xml',
'BIOMD0000000472_url.xml', 'BIOMD0000000245_url.xml',
'BIOMD0000000496_url.xml', 'BIOMD0000000473_url.xml',
'BIOMD0000000497_url.xml', 'BIOMD0000000248_url.xml',
'BIOMD0000000565_url.xml', 'BIOMD0000000364_url.xml',
'BIOMD0000000070_url.xml', 'BIOMD0000000232_url.xml',
'BIOMD0000000467_url.xml', 'BIOMD0000000383_url.xml',
'BIOMD0000000453_url.xml', 'BIOMD0000000112_url.xml',
'BIOMD0000000217_url.xml', 'BIOMD0000000452_url.xml',
'BIOMD0000000281_url.xml', 'BIOMD0000000439_url.xml',
'MODEL0072364382_url.xml', 'BIOMD0000000253_url.xml',
'BIOMD0000000243_url.xml', 'BIOMD0000000231_url.xml'
]

In [3]:
num = 3
simple = SimpleSBML()
simple.initialize(os.path.join(cn.DATA_DIR, multimulti_errors[num]))
s = StoichiometryMatrix(simple)
print(s.isConsistent())
m = MESGraph(simple)
m.analyze(simple.reactions)
print(m.type_three_errors)
#print(m.reduceReaction(simple.getReaction('re21')))
#print(m.processMultiMultiReaction(simple.getReaction('re21')))
print(m.processMultiMultiReaction(m.multimulti_reactions[0]))
print(m.type_three_errors)

False
No mass balance error found.
************************************
************************************
reduced_reaction re12: Glucose + PEP -> G6P + PYR
reduced_reaction re14: Pext + ATP -> 2.00 Pint + ADP
reduced_reaction re16: F6P + ATP -> FBP + ADP
reduced_reaction re18: G3P + Pint + NAD -> BPG + NADH
reduced_reaction re20: BPG + ADP -> PEP + ATP
reduced_reaction re21: PEP + ADP -> PYR + ATP
reduced_reaction re22: PYR + NADH -> Lactate + NAD
reduced_reaction re23: PYR + CoA -> AcetCoA + Formate
reduced_reaction re24: AcetCoA + 2.00 NADH -> Ethanol + 2.00 NAD + CoA
reduced_reaction re25: AcetCoA + ADP -> Acetate + ATP + CoA
reduced_reaction re27: Acetoin + NADH -> Butanediol + NAD
reduced_reaction re28: F6P + NADH -> Mannitol1Phosphate + NAD
reduced_reaction re30: PEP -> PYR
We don't have type III errors
We don't have type IV errors
[]
reduced_reaction re12: Glucose -> G6P
True
[]


In [7]:
simple = SimpleSBML()
simple.initialize(os.path.join(cn.DATA_DIR, multimulti_errors[num]))
simple.getReaction('re30')
print(m.nodes)

[{ATP}, {ADP}, {Pint}, {Pext}, {FBP}, {G3P}, {NAD}, {BPG}, {NADH}, {Lactate}, {CoA}, {AcetCoA}, {Formate}, {Ethanol}, {Acetate}, {Butanediol}, {Acetoin, Acetoin_Ext}, {Mannitol, Mannitol1Phosphate, Mannitol_Ext}, {PEP, PYR}, {F6P, G6P, Glucose}]


In [23]:
reaction = simple.getReaction('re12')
print(reaction.makeIdentifier(is_include_kinetics=False))
som = m.getNode(simple.getMolecule('PEP'))
reactants_in = collections.deque([mole_stoich for mole_stoich in  
                           reaction.reactants if 
                           m.getNode(mole_stoich.molecule)==som])
reactants_out = [mole_stoich for mole_stoich in  
              reaction.reactants if 
              m.getNode(mole_stoich.molecule)!=som]
products_in = collections.deque([mole_stoich for mole_stoich in  
                  reaction.products if 
                  m.getNode(mole_stoich.molecule)==som])
products_out = [mole_stoich for mole_stoich in  
             reaction.products if 
              m.getNode(mole_stoich.molecule)!=som]
print(reactants_in)
print(reactants_out)
print(products_in)
print(products_out)
while reactants_in and products_in:
  reactant = reactants_in[0]
  product = products_in[0]
  if reactant.stoichiometry > product.stoichiometry:
    reactants_in[0] = MoleculeStoichiometry(reactant.molecule,
                                          reactant.stoichiometry - product.stoichiometry)
    products_in.popleft()
  elif reactant.stoichiometry < product.stoichiometry:
    products_in[0] = MoleculeStoichiometry(product.molecule,
                                         product.stoichiometry - reactant.stoichiometry)
    reactants_in.popleft()
  else:
    reactants_in.popleft()
    products_in.popleft()
  reactants = list(reactants_in) + reactants_out
  products = list(products_in) + products_out
print(reactants)
print(products)
if (len(reaction.reactants) > len(reactants)) | \
    (len(reaction.products) > len(products)):
  reaction.reactants = reactants
  reaction.products = products
reaction.identifier = reaction.makeIdentifier()
reaction.category = reaction._getCategory() 
print(reaction)

re12: Glucose + PEP -> G6P + PYR
deque([PEP *  1.00])
[Glucose *  1.00]
deque([PYR *  1.00])
[G6P *  1.00]
[Glucose *  1.00]
[G6P *  1.00]
re12: Glucose -> G6P; function_4_PTS_Glc(FBP, G6P, Glucose, PEP, PYR, Pint, Vmax_PTS_Glc, kaPint_PTS_Glc, kiFBP_PTS_Glc, kmG6P_PTS_Glc, kmGlucose_PTS_Glc, kmPEP_PTS_Glc, kmPYR_PTS_Glc)


In [6]:
simple.getReaction('re12')

re12: Glucose -> G6P; function_4_PTS_Glc(FBP, G6P, Glucose, PEP, PYR, Pint, Vmax_PTS_Glc, kaPint_PTS_Glc, kiFBP_PTS_Glc, kmG6P_PTS_Glc, kmGlucose_PTS_Glc, kmPEP_PTS_Glc, kmPYR_PTS_Glc)

In [5]:
reaction = simple.getReaction('re12')
print(reaction)
reduced_reaction = m.reduceReaction(reaction)
reduced_reaction.makeIdentifier(is_include_kinetics=False)

re12: Glucose -> G6P; function_4_PTS_Glc(FBP, G6P, Glucose, PEP, PYR, Pint, Vmax_PTS_Glc, kaPint_PTS_Glc, kiFBP_PTS_Glc, kmG6P_PTS_Glc, kmGlucose_PTS_Glc, kmPEP_PTS_Glc, kmPYR_PTS_Glc)


AttributeError: 'bool' object has no attribute 'makeIdentifier'

In [12]:
m = MESGraph(simple)
m.analyze(simple.reactions)
print(m.nodes)
for r in simple.reactions:
  if r.category==cn.REACTION_n_n:
    print("we are processing", r.makeIdentifier(is_include_kinetics=False))
    print(m.processMultiMultiReaction(r))
print(m.type_three_errors)

No mass balance error found.
************************************
************************************
reduced_reaction re12: Glucose + PEP -> G6P + PYR
reduced_reaction re14: Pext + ATP -> 2.00 Pint + ADP
reduced_reaction re16: F6P + ATP -> FBP + ADP
reduced_reaction re18: G3P + Pint + NAD -> BPG + NADH
reduced_reaction re20: BPG + ADP -> PEP + ATP
reduced_reaction re21: PEP + ADP -> PYR + ATP
reduced_reaction re22: PYR + NADH -> Lactate + NAD
reduced_reaction re23: PYR + CoA -> AcetCoA + Formate
reduced_reaction re24: AcetCoA + 2.00 NADH -> Ethanol + 2.00 NAD + CoA
reduced_reaction re25: AcetCoA + ADP -> Acetate + ATP + CoA
reduced_reaction re27: Acetoin + NADH -> Butanediol + NAD
reduced_reaction re28: F6P + NADH -> Mannitol1Phosphate + NAD
We don't have type III errors
We don't have type IV errors
[{Glucose}, {ATP}, {ADP}, {Pint}, {Pext}, {FBP}, {G3P}, {NAD}, {BPG}, {NADH}, {Lactate}, {CoA}, {AcetCoA}, {Formate}, {Ethanol}, {Acetate}, {Butanediol}, {F6P, G6P}, {PEP, PYR}, {Acetoin,

In [43]:
m.type_three_errors

[PathComponents(node1={ADP}, node2={ATP}, reactions=['re21'])]

In [32]:
for r in simple.reactions:
  if r.category == cn.REACTION_n_n:
    print(r.makeIdentifier(is_include_kinetics=False))
    print(m.reduceReaction(r).makeIdentifier(is_include_kinetics=False))

re12: Glucose + PEP -> G6P + PYR
re12: Glucose -> G6P
re14: Pext + ATP -> 2.00 Pint + ADP
re14: Pext + ATP -> 2.00 Pint + ADP
re16: F6P + ATP -> FBP + ADP
re16: F6P + ATP -> FBP + ADP
re18: G3P + Pint + NAD -> BPG + NADH
re18: G3P + Pint + NAD -> BPG + NADH
re20: BPG + ADP -> PEP + ATP
re20: BPG + ADP -> PEP + ATP
re21: PEP + ADP -> PYR + ATP
re21: ADP -> ATP
re22: PYR + NADH -> Lactate + NAD
re22: PYR + NADH -> Lactate + NAD
re23: PYR + CoA -> AcetCoA + Formate
re23: PYR + CoA -> AcetCoA + Formate
re24: AcetCoA + 2.00 NADH -> Ethanol + 2.00 NAD + CoA
re24: AcetCoA + 2.00 NADH -> Ethanol + 2.00 NAD + CoA
re25: AcetCoA + ADP -> Acetate + ATP + CoA
re25: AcetCoA + ADP -> Acetate + ATP + CoA
re27: Acetoin + NADH -> Butanediol + NAD
re27: Acetoin + NADH -> Butanediol + NAD
re28: F6P + NADH -> Mannitol1Phosphate + NAD
re28: F6P + NADH -> Mannitol1Phosphate + NAD


In [39]:
adp = m.getNode(simple.getMolecule('ADP'))
atp = m.getNode(simple.getMolecule('ATP'))
# print(m.checkTypeThreeError(adp, atp, simple.getReaction('re21')))
# print(m.type_three_errors)

True
[PathComponents(node1={ADP}, node2={ATP}, reactions=['re21', 're21'])]


In [7]:
# num = -3
for er in multimulti_errors:
  simple = SimpleSBML()
  simple.initialize(os.path.join(cn.DATA_DIR, er))
  m = MESGraph(simple)
  m.analyze(error_details=False)

We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We have type III errors
 [PathComponents(node1={D, aS}, node2={S}, reactions=['SSAT_for_S']), PathComponents(node1={ORN, P, aD}, node2={D, aS}, reactions=['SSAT_for_D'])]
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We don't have type IV errors
We don't have type III errors
We 

KeyboardInterrupt: 

In [13]:
v2 = simple.getReaction('v2')
m.processMultiMultiReaction(v2)

True

In [15]:
m.type_three_errors

[]

In [125]:
def checkTypeThreeError(som1, som2, reaction):
  if m.has_edge(som1, som2):
    addTypeThreeError(som1, som2, reaction)
    return True
  elif m.has_edge(som2, som1):
    addTypeThreeError(som2, som1, reaction)
    return True
  return False

def mergeNodes(som1, som2, reaction):
  # Now mergeNodes assumes there is no error;
  # otherwise we would have already detected by checkTypeThreeError
  # (a) create a new SOM: som1.merge(som2)
  # (b) find edges starting or ending with som1/som2 and replace with this new SOM
  # delete som1 and som2 using remove_nodes_from
  new_som = som1.merge(som2)
  new_som.reactions.add(reaction)
  for som in [som1, som2]:
    for edge in list(m.in_edges(som)):
      remaining_som = edge[0]
      # add arc
      m.add_edge(remaining_som, new_som)  
    for edge in list(m.out_edges(som)):
      remaining_som = edge[1]
      # add arc
      m.add_edge(new_som, remaining_som) 
  m.remove_nodes_from([som1, som2])
  if not has_node(new_som):
    m.add_node(new_som)
  return m

In [78]:
def processMultiMultiReaction(reaction):
  pass
  # if all reactants are in one som, and all products are in one som
  # they must be the same (mergeSOM and update graph nodes & edges)
  flag = False
  reactants = [ms.molecule for ms in reaction.reactants]
  products = [ms.molecule for ms in reaction.products]
  reactant_soms = list({m.getNode(mole) for mole in reactants})
  product_soms = list({m.getNode(mole) for mole in products})
  print(reactant_soms)
  print(product_soms)
  if (len(reactant_soms)==1) and (len(product_soms)==1):
    flag = True
    # we should remove this multimulti reaction from self.multimulti_reactions
    reactant_stoichiometry = sum([ms.stoichiometry for ms in reaction.reactants])
    product_stoichiometry = sum([ms.stoichiometry for ms in reaction.products])
    if reactant_stoichiometry == product_stoichiometry:
        if reactant_soms[0] != product_soms[0]:
          if not checkTypeThreeError(som1, som2, reaction):
            mergeNodes(som1, som2, reaction)
        else:
          # this is obvious. pass (just add reaction)
          pass
    elif reactant_stoichciometry > product_stoichiometry:
      # meaning add arc from reactant_som -> product_som
      # check type I error and addArc
    else:
      # meaning add arc from product_som -> reactant_som
      # checck type I error and addArc
  else:
    # DON'T FORGET TO REMOVE THIS REACTION FROM SELF.MULTIMULTI_REACTIONS
    # now, we should reduce reaction
    # reduce reaction = namedtuple(label, reactants, products)
    # because of the above 
    # it doesn't merge soms but only add arcs (or type I errors)
    
    # flag = True if reduced reaction was used
    

  # elif, if both sides have the same som elements, reduce them by stoichiometry
  m.identifier = m.makeId()
  return flag

In [71]:
g = nx.DiGraph()
g.add_edges_from([('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'a'), ('a', 'e'), 
                  ('c', 'e'), ('f', 'c')])
# if remove 'a', all connected edges are deleted as well. But all nodes will still remain
g.remove_node('a')

In [72]:
print(g.edges)
print(g.nodes)

[('b', 'c'), ('c', 'd'), ('c', 'e'), ('f', 'c')]
['b', 'c', 'd', 'e', 'f']


In [73]:
a = [1, 2, 3, 4, 5]
a[1:] + [a[0]]

[2, 3, 4, 5, 1]

In [74]:
print("before", g.in_edges('c'))
for edge in list(g.in_edges('c')):
  remaining_som = edge[0]
  g.add_edge(remaining_som, 'z')
  g.remove_edge(edge[0], edge[1])
print("after", g.in_edges('z'))

before [('b', 'c'), ('f', 'c')]
after [('b', 'z'), ('f', 'z')]


In [75]:
g.edges

OutEdgeView([('b', 'z'), ('c', 'd'), ('c', 'e'), ('f', 'z')])

In [8]:
list(m.nodes)

[{Glc}, {HMP}, {Fru16P2}, {ADP, ATP}]

In [8]:
s.stoichiometry_matrix

,re12,re13,re14,re15,re16,re17,re18,re20,re21,re22,...,re24,re25,re26,re27,re28,re29,re30,re31,re32,re33
FBP,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
Formate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ADP,0.0,1.0,1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PEP,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
AcetCoA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Glucose,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NADH,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,...,-2.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
CoA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G3P,0.0,0.0,0.0,0.0,0.0,2.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pint,0.0,1.0,2.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
from scipy.optimize import linprog
import numpy as np
s_matrix_t = s.stoichiometry_matrix.T
# number of molecules
nmet = s_matrix_t.shape[0]
# number of reactions
nreac = s_matrix_t.shape[1]
#
b = np.zeros(nmet)
c = np.ones(nreac)
# Linear programming. c is constraint (here, zero), 
# b is vector of possible values for molecule vector. 
res = linprog(c, A_eq=s_matrix_t, b_eq=b, bounds=(1, None))
res.status

2